# Predicting links in Social Networks

In [162]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.utils import resample
import pickle

## Load data

### Like relation

In [18]:
like_1 = pd.read_csv("data/engineered_features/Like_Features_day_1.csv")
like_2 = pd.read_csv("data/engineered_features/Like_Features_day_2.csv")
like_3 = pd.read_csv("data/engineered_features/Like_Features_day_3.csv")
like_4 = pd.read_csv("data/engineered_features/Like_Features_day_4.csv")
like_5 = pd.read_csv("data/engineered_features/Like_Features_day_5.csv")

In [135]:
common_1 = pd.read_csv("data/engineered_features/common_interest_day_1.csv")
common_2 = pd.read_csv("data/engineered_features/common_interest_day_2.csv")
common_3 = pd.read_csv("data/engineered_features/common_interest_day_3.csv")
common_4 = pd.read_csv("data/engineered_features/common_interest_day_4.csv")
common_1_3 = pd.read_csv("data/engineered_features/common_interest_combined_day_1_3.csv")

### Group Relation

In [20]:
with open("data/engineered_features/G_features_postings_days_1-3.pkl", "rb") as input_file:
    G_1_3 = pickle.load(input_file)

### Posting TFIDF + graph features

In [58]:
with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_days_1to3.pkl", "rb") as input_file:
    tfidf_pos_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_day_4_test.pkl",
          "rb") as input_file:
    tfidf_pos_4_test = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_days_1to3.pkl", "rb") as input_file:
    tfidf_neg_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_day_4_test.pkl",
          "rb") as input_file:
    tfidf_neg_4_test = pickle.load(input_file)

## Process data for ML

### Downsample data

Experiment paramters:

In [189]:
TEST_SET_DOWN_FACTOR = 0.3
TRAIN_SET_DOWN_FACTOR = 0.1
TRAIN_SET_NEG_OVERBALANCE = 25
PREDICTION_GOAL = "DAY_4"
#PREDICTION_GOAL = "SAME_DAY"

#Exludes long-running classifiers
QUICK_RUN = True

Because of the huge number of nodes execution of basic data handling takes long on commodity hardware. Therefore the data sets need to be reduced in size while keeping the characteristics of the orignal data set

#### For the common interest relationship

In [190]:
if PREDICTION_GOAL == "SAME_DAY":
    com = common_1_3
    com["label"] = [1 if c > 0 else 0 for c in common_1_3["weight"]]
elif PREDICTION_GOAL == "DAY_4":
    com_4 = common_4[["source","target","weight"]]
    merged_com = pd.merge(common_1_3, com_4, on=["source","target"], how="left")
    merged_com.rename(columns= {"weight_y": "day_4_weight"}, inplace=True)
    merged_com['day_4_weight'] = merged_com['day_4_weight'].fillna(0)
    com = merged_com
    com["label"] = [1 if c > 0 else 0 for c in merged_com["day_4_weight"]]

In [191]:
com_all_downsample = resample(
    com,
    n_samples=int(com.shape[0] * TEST_SET_DOWN_FACTOR)
)

com_neg = com[com.label == 0]
com_pos = com[com.label == 1]

com_neg_downsample = resample(com_neg,
                              n_samples=int(com_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                              )

com_resampled = resample(
    pd.concat([com_pos, com_neg_downsample]),
    n_samples=int(com_pos.shape[0] * 2 * TRAIN_SET_DOWN_FACTOR)
)

In [192]:
com_resampled.shape[0]

7892

The resampled data structure still contains over 200k links (not considering train set downscaling), but now the label imbalance is corrected.

#### For the like relationship

In [193]:
if PREDICTION_GOAL == "SAME_DAY":
    like_all = pd.concat([like_1, like_2, like_3, like_4, like_5])
    like_all["label"] = [1 if c > 0 else 0 for c in like_all["current_weight"]]
elif PREDICTION_GOAL == "DAY_4":
    like_1_3 = pd.concat([like_1, like_2, like_3])

    like_4 = like_4[["source","target","current_weight"]]
    merged_like = pd.merge(like_1_3, like_4, on=["source","target"], how="left")
    merged_like.rename(columns= {"current_weight_y": "day_4_weight"}, inplace=True)
    merged_like['day_4_weight'] = merged_like['day_4_weight'].fillna(0)
    like_all = merged_like
    like_all["label"] = [1 if c > 0 else 0 for c in like_all["day_4_weight"]]

In [194]:
like_all

,day,source,target,jaccard_coef,adamic_adar_index,preferential_attachment_index,clustering_coefficient_score,current_weight_x,future_weight,day_4_weight,label
0,1,657,932,0.015873,0.229531,1015,0.000435,0,0,0.0,0
1,1,657,1277,0.014493,0.243257,1189,0.000631,0,0,0.0,0
2,1,657,1782,0.071429,1.254864,899,0.001344,0,0,0.0,0
3,1,657,2191,0.021277,0.192569,551,0.001386,0,0,0.0,0
4,1,657,2968,0.006993,0.192569,3335,0.001309,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
13404069,3,691789,690766,0.008969,0.452060,884,0.001575,0,0,0.0,0
13404070,3,691789,690933,0.142857,0.227560,16,0.013889,0,0,0.0,0
13404071,3,691789,691033,0.011905,0.224500,324,0.006723,-1,0,0.0,0
13404072,3,691789,691215,0.034483,0.227560,104,0.005897,0,0,0.0,0


In [195]:
like_all_downsampled = resample(
    like_all,
    n_samples=int(like_all.shape[0] * TEST_SET_DOWN_FACTOR)
)

like_neg = like_all[like_all.label == 0]
like_pos = like_all[like_all.label == 1]

like_neg_downsampled = resample(like_neg,
                                n_samples=int(like_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                                )
like_resampled = resample(
    pd.concat([like_pos, like_neg_downsampled]),
    n_samples=int(like_pos.shape[0] * 2 * TRAIN_SET_DOWN_FACTOR)
)

In [196]:
like_resampled.shape[0]

3889

The resampled data structure still contains over 400k (not considering train set downscaling) links, but now the label imbalance is corrected.

#### For group relation using TFIDF


In [198]:
if PREDICTION_GOAL == "SAME_DAY":
    tfidf_all_train = pd.concat([tfidf_neg_1_3, tfidf_pos_1_3])
    tfidf_all = tfidf_all_train
    tfidf_all_test = pd.concat([tfidf_neg_4_test, tfidf_pos_4_test])
elif PREDICTION_GOAL == "DAY_4":
    tfidf_1_3 = pd.concat([tfidf_neg_1_3, tfidf_pos_1_3])
    tfidf_4 = pd.concat([tfidf_neg_4_test, tfidf_pos_4_test])

    tfidf_4 = tfidf_4[["Source_User","Target_User","label"]]
    merged_tfidf = pd.merge(tfidf_1_3, tfidf_4, on=["Source_User","Target_User"], how="left")
    merged_tfidf.rename(columns= {"label_y": "label"}, inplace=True)
    merged_tfidf['label'] = merged_tfidf['label'].fillna(0)
    tfidf_all = merged_tfidf

In [200]:
tfidf_all_downsampled = resample(
    tfidf_all,
    n_samples=int(tfidf_all.shape[0] * TEST_SET_DOWN_FACTOR)
)

tfidf_neg = tfidf_all[tfidf_all.label == 0]
tfidf_pos = tfidf_all[tfidf_all.label == 1]

tfidf_neg_downsampled = resample(tfidf_neg,
                                n_samples=int(tfidf_pos.shape[0] * TRAIN_SET_NEG_OVERBALANCE)
                                )
tfidf_resampled = pd.concat([tfidf_pos, tfidf_neg_downsampled])

In [201]:
tfidf_neg_downsampled.shape[0]

85850

The resampled data structure still contains over 90k (not considering train set downscaling) links, but now the label imbalance is corrected.

## Define columns for training

In [202]:
target = "label"
graph_features = [
    "jaccard_coef",
    "adamic_adar_index",
    "preferential_attachment_index",
    "clustering_coefficient_score"
]
larger_graph_features = [
    "jaccard_coef",
    "adamic_adar_index",
    "preferential_attachment_index",
    "clustering_coefficient_score_Source_User",
    "clustering_coefficient_score_Target_User",
    "pagerank_Source_User",
    "pagerank_Target_User"
]
tfidf_features = [ f'{i}_TFIDF' for i in range(0,500)]


X_train_com = com_resampled[graph_features]
y_train_com = com_resampled[target]

X_train_like = like_resampled[graph_features]
y_train_like = like_resampled[target]

X_train_tfidf = tfidf_resampled[larger_graph_features + tfidf_features]
y_train_tfidf = tfidf_resampled[target]

In [203]:
rfc_com = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_com = SVC()
knn_com = KNeighborsClassifier(n_neighbors=10)

rfc_like = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_like = SVC()
knn_like = KNeighborsClassifier(n_neighbors=10)

rfc_tfidf = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_tfidf = SVC()
knn_tfidf = KNeighborsClassifier(n_neighbors=10)

## Define evaluation function

In [204]:
def evaluate_model(predictions, actual):
    return pd.DataFrame({
        "Measure": ["Accuracy", "Precision", "Recall", "F1"],
        "Score": [accuracy_score(actual, predictions),
                  precision_score(actual, predictions),
                  recall_score(actual, predictions),
                  f1_score(actual, predictions)]
    }).transpose()

## Fit classifiers

### RF

In [205]:
rfc_com.fit(X_train_com, y_train_com)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

In [206]:
rfc_like.fit(X_train_like, y_train_like)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

In [207]:
rfc_tfidf.fit(X_train_tfidf, y_train_tfidf)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

### SVM

In [208]:
if not QUICK_RUN:
    svm_com.fit(X_train_com, y_train_com)

In [209]:
if not QUICK_RUN:
    svm_like.fit(X_train_like, y_train_like)

In [210]:
if not QUICK_RUN:
    svm_tfidf.fit(X_train_tfidf, y_train_tfidf)

### KNN

In [211]:
knn_com.fit(X_train_com, y_train_com)

KNeighborsClassifier(n_neighbors=10)

In [212]:
knn_like.fit(X_train_like, y_train_like)

KNeighborsClassifier(n_neighbors=10)

In [213]:
knn_tfidf.fit(X_train_tfidf, y_train_tfidf)

KNeighborsClassifier(n_neighbors=10)

## Evaluate predictions

In [214]:
X_test_com = com_all_downsample[graph_features]
y_test_com = com_all_downsample[target]

In [215]:
X_test_like = like_all_downsampled[graph_features]
y_test_like = like_all_downsampled[target]

In [216]:
if PREDICTION_GOAL == "SAME_DAY":
    X_test_tfidf = tfidf_all_test[larger_graph_features + tfidf_features]
    y_test_tfidf = tfidf_all_test[target]
elif PREDICTION_GOAL == "DAY_4":
    X_test_tfidf = tfidf_all_downsampled[larger_graph_features + tfidf_features]
    y_test_tfidf = tfidf_all_downsampled[target]

### RF evaluation

In [217]:
predictions = rfc_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.989547,0.061299,0.302741,0.101954


In [218]:
predictions = rfc_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.997249,0.012251,0.011697,0.011967


In [219]:
predictions = rfc_tfidf.predict(X_test_tfidf)
evaluate_model(predictions, y_test_tfidf)

,0,1,2,3
Measure,Accuracy,Precision,Recall,F1
Score,0.997541,0.988108,0.879692,0.930754


### SVM evaluation

In [220]:
if not QUICK_RUN:
    predictions = svm_com.predict(X_test_com)
    evaluate_model(predictions, y_test_com)

In [221]:
if not QUICK_RUN:
    predictions = svm_like.predict(X_test_like)
    evaluate_model(predictions, y_test_like)

In [222]:
if not QUICK_RUN:
    predictions = svm_tfidf.predict(X_test_tfidf)
    evaluate_model(predictions, y_test_tfidf)

### KNN evaluation

In [ ]:
predictions = knn_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

In [ ]:
predictions = knn_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

In [ ]:
predictions = knn_tfidf.predict(X_test_tfidf)
evaluate_model(predictions, y_test_tfidf)

# TODO

- Split according to days (one day for test one for verfication?)
- Combine different features
- Visuailze predited networks
- Reformulate learning taks to predict the next day
- Grid search for params
- Cross validtation
- Explainability?


See this [repo](https://github.com/neo4j-examples/link-prediction) for reference